In [1]:
import sys
from tqdm import tqdm
sys.path.insert(0, '../')

import torch
from torch import nn
from torch.nn import functional as F

from transformer.transformer import MultiHeadAttention

ModuleNotFoundError: No module named 'transformer.transformer'; 'transformer' is not a package

In [17]:
pad_id = 0
vocab_size = 100
max_len = 10
hidden_dim = 6

def padding(data: list, pad_id: int=0) -> (list, int):
    max_len = len(max(data, key=len))
    output = [sample + [pad_id]*(max_len-len(sample)) for sample in tqdm(data)]
    return output, max_len

data = [
     [62, 13, 47, 39, 78, 33, 56, 13, 39, 29],
     [60, 96, 51, 32, 90, 44, 86, 71, 36, 18],
     [35, 45, 48, 65, 91, 99, 92, 10, 31, 21],
     [75, 51, 45, 48, 65, 91, 99, 11, 13, 28],
     [66, 88, 98, 47, 48, 65, 17, 13, 67, 99],
     ]

# data, max_len = padding(data)
data = torch.LongTensor(data)
print(data.shape)

torch.Size([5, 10])


In [18]:
embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=hidden_dim)

# w_query = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
# w_key = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
# w_value = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)

# embedding.weight.data.uniform_(-1, 1)
# w_query.weight.data.uniform_(-1, 1)
# w_key.weight.data.uniform_(-1, 1)
# w_value.weight.data.uniform_(-1, 1)
print('Initialized')

Initialized


In [19]:
embedding(data)

tensor([[[ 0.3500, -0.7940, -0.2843, -0.7397,  1.4171, -0.4520],
         [-0.6861, -2.8138,  0.3442,  0.2728, -0.6883,  1.1423],
         [ 0.3644, -1.2988, -0.0406,  0.6302, -0.0397,  1.1577],
         [ 0.7950, -1.6610,  0.0899, -0.9438,  1.3486,  0.5950],
         [-0.5862, -0.1035,  0.7650, -1.4547, -0.9456, -1.1637],
         [ 1.5435, -0.7741, -0.9196, -1.0213,  0.6943,  0.4392],
         [-0.4319,  0.2256, -1.2074,  1.0227, -1.4463, -0.9066],
         [-0.6861, -2.8138,  0.3442,  0.2728, -0.6883,  1.1423],
         [ 0.7950, -1.6610,  0.0899, -0.9438,  1.3486,  0.5950],
         [ 0.3734,  0.5410, -1.4976, -0.3848,  1.5991,  1.7465]],

        [[-0.3957,  0.6645, -0.1004, -0.6024, -1.7313,  0.5948],
         [ 0.0749,  0.0932, -1.1894, -1.6060, -0.1776,  1.7808],
         [ 0.2848, -1.0590,  0.5895,  0.5058,  0.9810, -0.7540],
         [ 0.3156,  0.8662, -0.5489, -0.5073,  3.8343, -0.6247],
         [ 0.5189,  0.6964, -0.5791, -0.0510, -0.3047,  0.4854],
         [ 0.7041, -1.3

In [99]:
embedding(data).shape # (batgch_size, seq_length, max_length, embedding_dim)

torch.Size([5, 10, 6])

In [25]:
batch_size = 1
max_len = 4
num_heads = 2
d_k = 3
q = torch.ones(batch_size, max_len, num_heads, d_k)
k = torch.ones(batch_size, max_len, num_heads, d_k)
q, k

(tensor([[[[1., 1., 1.],
           [1., 1., 1.]],
 
          [[1., 1., 1.],
           [1., 1., 1.]],
 
          [[1., 1., 1.],
           [1., 1., 1.]],
 
          [[1., 1., 1.],
           [1., 1., 1.]]]]),
 tensor([[[[1., 1., 1.],
           [1., 1., 1.]],
 
          [[1., 1., 1.],
           [1., 1., 1.]],
 
          [[1., 1., 1.],
           [1., 1., 1.]],
 
          [[1., 1., 1.],
           [1., 1., 1.]]]]))